https://www.analyticsvidhya.com/blog/2021/10/beginners-guide-to-automl-with-an-easy-autogluon-example/

In [1]:
import pandas as pd

from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/strokes/stroke-real.csv')

In [3]:
df.drop(columns='id', inplace=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df_train, df_test=train_test_split(df, test_size=0.4, random_state=42)
df_train.shape,df_test.shape

((3066, 11), (2044, 11))

In [5]:
predictor= TabularPredictor(
    label='stroke',
    problem_type='binary',
    eval_metric='accuracy',
    path='AutogluonModels/strokes/real/v1',
    learner_kwargs = {'positive_class':1}
    )

In [6]:
predictor.fit(
    train_data = df_train,
    verbosity = 2,
    presets=['best_quality'],
    num_gpus=1
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       7.10 GB / 15.71 GB (45.2%)
Disk Space Avail:   670.61 GB / 933.46 GB (71.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up 

In [7]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                          model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   NeuralNetFastAI_r191_BAG_L2   0.957926    accuracy       3.728564  1772.997116                0.185516          57.975660            2       True         53
1           WeightedEnsemble_L3   0.957926    accuracy       3.889835  1988.088516                0.000000           0.260479            3       True         56
2     NeuralNetTorch_r79_BAG_L2   0.957926    accuracy       3.889835  1987.828037                0.346787         272.806581            2       True         51
3           WeightedEnsemble_L2   0.957599    accuracy       0.068066     1.517360                0.000000           0.286325            2       True         38
4          LightGBM_r130_BAG_L1   0.957599    accuracy       0.068066     1.231036                0.068066           1.231036        

c:\Users\jgala\Documents\uned\tfm\synthetic-data\.venv\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetTorch_r79_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiT

In [8]:
predictor.evaluate(df_test)

{'accuracy': 0.9452054794520548,
 'balanced_accuracy': 0.5087719298245614,
 'mcc': 0.12876960330741333,
 'roc_auc': 0.8265157712935188,
 'f1': 0.034482758620689655,
 'precision': 1.0,
 'recall': 0.017543859649122806}

In [9]:
predictor.leaderboard(df_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,1.000000,0.951729,accuracy,0.038627,0.043877,0.000000,0.038627,0.043877,0.000000,1,True,2
1,ExtraTreesEntr_BAG_L1,1.000000,0.954338,accuracy,0.110981,0.099566,0.546892,0.110981,0.099566,0.546892,1,True,9
2,RandomForestEntr_BAG_L1,1.000000,0.955316,accuracy,0.119904,0.100333,0.592280,0.119904,0.100333,0.592280,1,True,6
3,RandomForestGini_BAG_L1,1.000000,0.954664,accuracy,0.124974,0.118347,1.057970,0.124974,0.118347,1.057970,1,True,5
4,RandomForest_r195_BAG_L1,1.000000,0.952381,accuracy,0.126321,0.100924,0.978678,0.126321,0.100924,0.978678,1,True,26
5,ExtraTrees_r42_BAG_L1,1.000000,0.953686,accuracy,0.128382,0.114018,0.679440,0.128382,0.114018,0.679440,1,True,22
6,ExtraTreesGini_BAG_L1,1.000000,0.954012,accuracy,0.134038,0.090709,0.572472,0.134038,0.090709,0.572472,1,True,8
7,CatBoost_r9_BAG_L1,0.962492,0.956621,accuracy,0.047792,0.018091,60.139861,0.047792,0.018091,60.139861,1,True,18
8,CatBoost_r13_BAG_L1,0.960535,0.956621,accuracy,0.031563,0.051726,14.478398,0.031563,0.051726,14.478398,1,True,25
9,KNeighborsUnif_BAG_L1,0.960535,0.953033,accuracy,0.032932,0.051343,0.000000,0.032932,0.051343,0.000000,1,True,1


In [10]:
predictor.feature_importance(data=df_train)

Computing feature importance via permutation shuffling for 10 features using 3066 rows with 5 shuffle sets...
	478.66s	= Expected runtime (95.73s per shuffle set)
	285.46s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
age,0.002414,0.000592,0.000403,5,0.003634,0.001194
bmi,0.002348,0.000425,0.000124,5,0.003224,0.001473
avg_glucose_level,0.001957,0.000231,0.000023,5,0.002432,0.001482
smoking_status,0.001239,0.000536,0.003323,5,0.002343,0.000136
Residence_type,0.001174,0.000438,0.001941,5,0.002075,0.000273
ever_married,0.000783,0.000292,0.001941,5,0.001383,0.000182
gender,0.000783,0.000592,0.020897,5,0.002003,-0.000437
hypertension,0.000522,0.000179,0.001419,5,0.000890,0.000154
work_type,0.000522,0.000592,0.060122,5,0.001742,-0.000698
heart_disease,0.000196,0.000292,0.104000,5,0.000796,-0.000405
